# Fermi GBM Detector Geometry and Selection

This notebook demonstrates how to use `grb_pipeline` to calculate the angle between
a GRB position and each Fermi GBM detector, and automatically select the optimal
detectors for spectral analysis.

The coordinate transformation code follows Vianello's method from
[gtburst](https://github.com/giacomov/gtburst).

Authors: Vikas Chand

In [ ]:
import numpy as np
from astropy.io import fits
import os
from grb_pipeline.utils.gbm_geometry import (
    get_ra_dec, get_theta_phi, get_detector_angle,
    get_all_detector_angles, select_gbm_detectors, Vector,
    GBM_DETECTOR_ANGLES
)

## 1. Read GRB Position and Spacecraft Pointing from FITS

For a Fermi/GBM triggered dataset, the spacecraft pointing (SCX, SCZ) 
is stored in the trigdat file, and the GRB position (RA_OBJ, DEC_OBJ)
is in the response (rsp) files.

In [ ]:
# Example: GRB 080723913
# These values would normally come from the FITS headers
# RA_OBJ and DEC_OBJ from the rsp file:
RA_OBJ = 113.3
DEC_OBJ = -19.7

# Spacecraft pointing from the trigdat file:
RA_SCX = 153.562  # Spacecraft X-axis RA
DEC_SCX = -25.339  # Spacecraft X-axis Dec
RA_SCZ = 83.891   # Spacecraft Z-axis RA
DEC_SCZ = 30.471   # Spacecraft Z-axis Dec

print(f"GRB position: RA={RA_OBJ}, Dec={DEC_OBJ}")
print(f"Spacecraft X: RA={RA_SCX}, Dec={DEC_SCX}")
print(f"Spacecraft Z: RA={RA_SCZ}, Dec={DEC_SCZ}")

## 2. Calculate Angles to All Detectors

Using the spacecraft pointing and the GRB position, we calculate
the angle from the source to each of the 14 GBM detectors
(12 NaI + 2 BGO).

In [ ]:
# Calculate angles to all 14 detectors
sorted_detectors = get_all_detector_angles(
    RA_SCX, DEC_SCX, RA_SCZ, DEC_SCZ, RA_OBJ, DEC_OBJ
)

print("GBM detectors sorted by angle to source:\n")
print(f"{'Detector':>10} {'Angle (deg)':>12}")
print("-" * 24)
for det_name, angle in sorted_detectors:
    print(f"{det_name:>10} {angle:>12.3f}")

## 3. Coordinate Transformations

The `get_theta_phi` function converts celestial coordinates (RA, Dec) to 
spacecraft coordinates (theta, phi), and `get_ra_dec` does the reverse.

In [ ]:
# Convert GRB RA/Dec to spacecraft theta/phi
theta, phi = get_theta_phi(RA_SCX, DEC_SCX, RA_SCZ, DEC_SCZ, RA_OBJ, DEC_OBJ)
print(f"GRB in spacecraft coordinates: theta={theta:.2f} deg, phi={phi:.2f} deg")

# Verify: convert back to RA/Dec
ra_check, dec_check = get_ra_dec(RA_SCX, DEC_SCX, RA_SCZ, DEC_SCZ, theta, phi)
print(f"Converted back: RA={ra_check:.3f}, Dec={dec_check:.3f}")
print(f"Original:       RA={RA_OBJ:.3f}, Dec={DEC_OBJ:.3f}")

## 4. Automatic Detector Selection

`select_gbm_detectors` automatically chooses the best NaI and BGO detectors
based on source angle. Detectors within 60° are considered, and the BGO
facing the selected NaI cluster is chosen.

In [ ]:
# Automatic detector selection using SCX/SCZ method
selection = select_gbm_detectors(
    source_ra=RA_OBJ,
    source_dec=DEC_OBJ,
    ra_scx=RA_SCX,
    dec_scx=DEC_SCX,
    ra_scz=RA_SCZ,
    dec_scz=DEC_SCZ,
)

print(f"Selected NaI detectors: {selection['nai_detectors']}")
print(f"Selected BGO detectors: {selection['bgo_detectors']}")
print(f"\nAll angles:")
for det, angle in sorted(selection['angles'].items(), key=lambda x: x[1]):
    marker = " <-- selected" if det in selection['nai_detectors'] + selection['bgo_detectors'] else ""
    print(f"  {det}: {angle:.1f} deg{marker}")

## 5. Individual Detector Angle Calculation

You can also compute the angle for a specific detector.

In [ ]:
# Calculate angle for a specific detector
for det in ['n0', 'n6', 'b0', 'b1']:
    angle = get_detector_angle(RA_SCX, DEC_SCX, RA_SCZ, DEC_SCZ, RA_OBJ, DEC_OBJ, det)
    print(f"Detector {det}: {angle:.3f} deg")